<a href="https://colab.research.google.com/github/fatrrr/PANTURA-C23-PC596_machine_learning/blob/main/road_damage_detection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    %tensorflow_version 2.x
except:
    pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib as plt
import os
import cv2
import sklearn
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
#Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''
import zipfile
zip_dir = "/content/drive/Shareddrives/Pantura/colab_notebook/road_damage_prioritize_dataset.zip"
destination_dir= "/content/drive/Shareddrives/Pantura/colab_notebook/"

with zipfile.ZipFile(zip_dir, 'r') as zip_ref:
  zip_ref.extractall(destination_dir)

zip_ref.close()
'''

'\nimport zipfile\nzip_dir = "/content/drive/Shareddrives/Pantura/colab_notebook/road_damage_prioritize_dataset.zip"\ndestination_dir= "/content/drive/Shareddrives/Pantura/colab_notebook/"\n\nwith zipfile.ZipFile(zip_dir, \'r\') as zip_ref:\n  zip_ref.extractall(destination_dir)\n\nzip_ref.close()\n'

**Transfer Learning**

In [ ]:
'''
m = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1",
                   trainable=False),  # Can be True, see below.
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
m.build([None, expect_img_size, expect_img_size, 3])  # Batch input shape.
'''

'\nm = tf.keras.Sequential([\n\xa0 \xa0 hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1",\n\xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0   \xa0 \xa0trainable=False), \xa0# Can be True, see below.\n\xa0 \xa0 tf.keras.layers.Dense(num_classes, activation=\'softmax\')\n])\nm.build([None, expect_img_size, expect_img_size, 3]) \xa0# Batch input shape.\n'

In [ ]:
MODULE_HANDLE = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"
pixels = 224
FV_SIZE = 1280
IMAGE_SIZE =(pixels, pixels)

feature_extractor = hub.KerasLayer(MODULE_HANDLE,
                                   input_shape=IMAGE_SIZE + (3,),
                                   output_shape=[FV_SIZE],
                                   trainable=False)

In [ ]:
'''NUM_LAYERS = 10

if do_fine_tuning:
    feature_extractor.trainable = True

    for layer in model.layers[-NUM_LAYERS:]:
        layer.trainable = True

else:
    feature_extractor.trainable = False'''

'NUM_LAYERS = 10\n\nif do_fine_tuning:\n    feature_extractor.trainable = True\n\n    for layer in model.layers[-NUM_LAYERS:]:\n        layer.trainable = True\n\nelse:\n    feature_extractor.trainable = False'

# Image Processing

In [ ]:
data_dir =  "/content/drive/Shareddrives/Pantura/colab_notebook/sih_road_dataset"
labels = ["tidak_ada_kerusakan", "rendah", "sedang", "tinggi"]
x = []
y = []
for label in labels:
    data = os.path.join(data_dir,label)
    for image in os.listdir(data):
        try:
            img = cv2.imread(os.path.join(data, image), cv2.IMREAD_COLOR)
            img = cv2.resize(img, IMAGE_SIZE)

            x.append(img)
            y.append(labels.index(label))

        except Exception as e:
            pass

In [ ]:
np.unique(y)
np.unique(x)

x = np.array(x)/255.0
y = np.array(y)

x.shape
y.shape

x = x.reshape(-1, pixels, pixels, 3)
x.shape

y = y.reshape(-1, 1)

In [ ]:
y = tf.keras.utils.to_categorical(y, 4)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagenerator = ImageDataGenerator(
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.1,
      zoom_range=0.1,
      horizontal_flip=False,
      vertical_flip=False,
      fill_mode='nearest')

datagenerator.fit(x)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [ ]:
train_generator = datagenerator.flow(x_train, y_train, batch_size=32)
val_generator = datagenerator.flow(x_test, y_test)

# Model

**Model**

In [ ]:
model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(4, activation='softmax'),
])

#Load Weight
model.load_weights('/content/drive/Shareddrives/Pantura/colab_notebook/checkpoints/')

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              4049564   
                                                                 
 dense (Dense)               (None, 4)                 5124      
                                                                 
Total params: 4,054,688
Trainable params: 5,124
Non-trainable params: 4,049,564
_________________________________________________________________


In [ ]:
'''earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)


y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

y_true = np.argmax(y_test, axis=1)

recall = recall_score(y_true, y_pred, average='weighted')

precision = precision_score(y_true, y_pred, average='weighted')

f1 = f1_score(y_true, y_pred, average='weighted')

print('Recall:', recall)
print('Precision:', precision)
print('F1-score:', f1)

model.fit(
    train_generator,
    epochs=1,
    callbacks=[earlystop],
    validation_data=val_generator
)
'''

"earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)\n\n\ny_pred = model.predict(x_test)\ny_pred = np.argmax(y_pred, axis=1)\n\ny_true = np.argmax(y_test, axis=1)\n\nrecall = recall_score(y_true, y_pred, average='weighted')\n\nprecision = precision_score(y_true, y_pred, average='weighted')\n\nf1 = f1_score(y_true, y_pred, average='weighted')\n\nprint('Recall:', recall)\nprint('Precision:', precision)\nprint('F1-score:', f1)\n\nmodel.fit(\n    train_generator,\n    epochs=1,\n    callbacks=[earlystop],\n    validation_data=val_generator\n)\n"

In [ ]:
#Save weight

#weight_path_ckpt = "/content/drive/Shareddrives/Pantura/colab_notebook/checkpoints/"
#model.save_weights(weight_path_ckpt)

In [ ]:
#tf.keras.backend.clear_session()

In [ ]:
#tensorboard
'''import datetime

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(
    train_generator,
    epochs=10,
    callbacks=[tensorboard_callback],
    validation_data=val_generator
)'''

'import datetime\n\nlog_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")\n\ntensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)\n\nmodel.fit(\n    train_generator,\n    epochs=10,\n    callbacks=[tensorboard_callback],\n    validation_data=val_generator\n)'

In [ ]:
#!tensorboard dev upload --logdir ./logs

# Saving Model



In [ ]:
'''
import pathlib

# export the savedmodel
export_dir = '/content/drive/Shareddrives/Pantura/colab_notebook/saved_model'
tf.saved_model.save(model, export_dir)

# convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

#save the model
tflite_model_file = pathlib.Path('/content/drive/Shareddrives/Pantura/colab_notebook/tflite_model/foo.tflite')
tflite_model_file.write_bytes(tflite_model)
'''

"\nimport pathlib\n\n# export the savedmodel\nexport_dir = '/content/drive/Shareddrives/Pantura/colab_notebook/saved_model'\ntf.saved_model.save(model, export_dir)\n\n# convert the model\nconverter = tf.lite.TFLiteConverter.from_saved_model(export_dir)\ntflite_model = converter.convert()\n\n#save the model\ntflite_model_file = pathlib.Path('/content/drive/Shareddrives/Pantura/colab_notebook/foo.tflite')\ntflite_model_file.write_bytes(tflite_model)\n"

In [ ]:
# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
#model.save('my_model.h5')

In [ ]:
#model.json
#!pip install tensorflowjs

#!tensorflowjs_converter \
#    --input_format=keras \
#    /content/my_model.h5 \
#    /content/drive/Shareddrives/Pantura/colab_notebook/js_model

In [ ]:
#!pip install tflite-support
#!pip install absl-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for tflite-support: filename=tflite_support-0.1.0a1-cp310-cp310-linux_x86_64.whl size=5942594 sha256=acf53d2f560f2e6a77f683c87ed46ce70f874edbaf43111559b2f2ea142fcb3c
  Stored in directory: /root/.cache/pip/wheels/71/5c/da/9e5e661ec26e03ee57e69428d40fffbefe3c0aff649c55776d
Successfully built tflite-support
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#create metadata
'''
from tflite_support import flatbuffers
from tflite_support import metadata as _metadata
from tflite_support import metadata_schema_py_generated as _metadata_fb

""" ... """
"""Creates the metadata for an image classifier."""

# Creates model info.
model_meta = _metadata_fb.ModelMetadataT()
model_meta.name = "PanturaV1 road damage classifier"
model_meta.description = ("Identify 4 different levels of road damages")
model_meta.version = "v1"
model_meta.author = "Pantura"
model_meta.license = ("Apache License. Version 2.0 "
                      "http://www.apache.org/licenses/LICENSE-2.0.")

# Creates input info.
input_meta = _metadata_fb.TensorMetadataT()

# Creates output info.
output_meta = _metadata_fb.TensorMetadataT()

input_meta.name = "image"
input_meta.description = (
    "Input image to be classified. The expected image is {0} x {1}, with "
    "three channels (red, blue, and green) per pixel. Each value in the "
    "tensor is a single byte between 0 and 255.".format(224, 224))
input_meta.content = _metadata_fb.ContentT()
input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
input_meta.content.contentProperties.colorSpace = (
    _metadata_fb.ColorSpaceType.RGB)
input_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.ImageProperties)
input_normalization = _metadata_fb.ProcessUnitT()
input_normalization.optionsType = (
    _metadata_fb.ProcessUnitOptions.NormalizationOptions)
input_normalization.options = _metadata_fb.NormalizationOptionsT()
input_normalization.options.mean = [127.5]
input_normalization.options.std = [127.5]
input_meta.processUnits = [input_normalization]
input_stats = _metadata_fb.StatsT()
input_stats.max = [255]
input_stats.min = [0]
input_meta.stats = input_stats

# Creates output info.
output_meta = _metadata_fb.TensorMetadataT()
output_meta.name = "probability"
output_meta.description = "Probabilities of the 4 labels respectively."
output_meta.content = _metadata_fb.ContentT()
output_meta.content.content_properties = _metadata_fb.FeaturePropertiesT()
output_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_stats = _metadata_fb.StatsT()
output_stats.max = [1.0]
output_stats.min = [0.0]
output_meta.stats = output_stats
label_file = _metadata_fb.AssociatedFileT()
label_file.name = os.path.basename("/content/drive/Shareddrives/Pantura/colab_notebook/tflite_model/label.txt")
label_file.description = "Labels for 4 levels of road damages that the model can recognize."
label_file.type = _metadata_fb.AssociatedFileType.TENSOR_AXIS_LABELS
output_meta.associatedFiles = [label_file]

# Creates subgraph info.
subgraph = _metadata_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]
subgraph.outputTensorMetadata = [output_meta]
model_meta.subgraphMetadata = [subgraph]

b = flatbuffers.Builder(0)
b.Finish(
    model_meta.Pack(b),
    _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = b.Output()



populator = _metadata.MetadataPopulator.with_model_file("/content/drive/Shareddrives/Pantura/colab_notebook/tflite_model/foo.tflite")
populator.load_metadata_buffer(metadata_buf)
#populator.load_associated_files(["/content/drive/Shareddrives/Pantura/colab_notebook/tflite_model/label.txt"])
populator.populate()

displayer = _metadata.MetadataDisplayer.with_model_file("/content/drive/Shareddrives/Pantura/colab_notebook/tflite_model/foo.tflite")
export_json_file = os.path.join("/content/drive/Shareddrives/Pantura/colab_notebook/tflite_model/",
                                os.path.splitext("foo_metadata")[0] + ".json")
json_file = displayer.get_metadata_json()
# Optional: write out the metadata as a json file
with open(export_json_file, "w") as f:
  f.write(json_file)
'''

/usr/lib/python3.10/zipfile.py:1519: UserWarning: Duplicate name: 'colab_notebook/'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
